In [1]:
%reset -f

In [2]:
import ee
ee.Authenticate()


In [ ]:

ee.Initialize(project='ee-andrewfullhart')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

ndays_months = ee.List([31, 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])

def periodfn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  period_ic = nex_ic.filterDate(start, end)
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  period_ic = period_ic.filter(modelfilter)
  period_ic = period_ic.filter(ee.Filter.eq('model', climate_model))
  period_ic_list = period_ic.toList(999)

  def getImageDate(imobj):
      return im.date()

  def month_fn(mobj):
    m = ee.Number(mobj)
    pr_im = year_ic_list.get(m.subtract(1)).select('pr').multiply(86400).multiply(ee.Number(ndays_months.get(m.subtract(1))))
    tx_im = ee.Image(year_ic_list.get(m.subtract(1)).select('tasmax').subtract(273.15)).multiply(ee.Number(ndays_months.get(m.subtract(1)))).divide(365.25)
    tn_im = ee.Image(year_ic_list.get(m.subtract(1)).select('tasmin').subtract(273.15)).multiply(ee.Number(ndays_months.get(m.subtract(1)))).divide(365.25)
    return pr_im.addBands(tx_im).addBands(tn_im)
  converted_ic = ee.ImageCollection(order_months.map(month_fn))
  sum_im = converted_ic.reduce(ee.Reducer.sum()).clip(area_shp)
  return null

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='accm_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1